In [ ]:
TEST_RATIO = 1

In [ ]:
SAMPLE_INTERVAL = 60
SAMPLE_HISTORY_FILENAME = 'sample-history.txt'
TRANSACTION_HISTORY_FILENAME = 'transaction-history.txt'

In [ ]:
WEBHOOK_URL = 'https://hooks.slack.com/services/*********/*********/************************'
CHANNEL_ID = '*********'

In [ ]:
def send_slack(message, text, color='good'):
    if color == 'good':
        message = f':star-struck: {message}'
    if color == 'danger':
        message = f':rage: {message}'

    import requests
    requests.post(
        WEBHOOK_URL,
        json={
            'channel': CHANNEL_ID,
            'username': 'Trading Notifications',
            'icon_url': 'https://ethereum.org/favicon-32x32.png',
            'text': message,
            'mrkdwn': True,
            'attachments': [{
                'title': '→ Trading Operations',
                'title_link': 'https://jhub.name/user/sheng_wei/lab',
                'text': text,
                'color': color,
            }]
        }
    )

In [ ]:
UPPER_TRESHOLD = 1000
BASE_TRESHOLD = 0

In [ ]:
import re, time
with open(SAMPLE_HISTORY_FILENAME, 'r') as fsh, open(TRANSACTION_HISTORY_FILENAME, 'r') as fth:
    num = 1
    print('\n', end='')
    while True:
        sample_lines = fsh.readlines()[-3:-1]

        res = None
        for line in sample_lines:
            res = re.search(r'^GAIN ((?:-|\+)\d+\.?\d*)=.+ JPY in total at (.+)\n', line)
            if res is not None:
                break
        if res is None:
            time.sleep(SAMPLE_INTERVAL/TEST_RATIO/10)
            continue

        gain_jpy = float(res.group(1))

        report_markdown = ''
        samples = ''.join(sample_lines)
        if samples != '':
            report_markdown = f'{report_markdown}\n```\n{samples}```'
            samples = f'{samples}\n'
        transactions = ''.join(fth.readlines()[-3:-1])
        if transactions != '':
            report_markdown = f'\n{report_markdown}\n```\n{transactions}```'
            transactions = f'{transactions}\n'

        if gain_jpy > UPPER_TRESHOLD:
            send_slack(
                f'Total Gain `{gain_jpy} JPY` > `{UPPER_TRESHOLD} JPY`: NO.{num} @ `{res.group(2)}`',
                f'{report_markdown}\nPower by https://jhub.name/',
                'good'
            )
        if gain_jpy < BASE_TRESHOLD:
            send_slack(
                f'Total Gain `{gain_jpy} JPY` < `{BASE_TRESHOLD} JPY`: NO.{num} @ `{res.group(2)}`',
                f'{report_markdown}\nPower by https://jhub.name/',
                'danger'
            )

        print(f'NO.{num}: {samples}{transactions}', end='')
        time.sleep(SAMPLE_INTERVAL / TEST_RATIO / 2)
        num += 1